In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.transforms as T
import torch_cluster
import torch_sparse

from torch_geometric.nn import Node2Vec
from torch_geometric.data import Data, DataLoader
from torch_geometric.utils import dropout_adj, negative_sampling, remove_self_loops,add_self_loops

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
ppi = torch.load("ppi.pt").cuda()

In [4]:
ppi

tensor(indices=tensor([[    0,     1,   322,  ..., 11393,  1356, 11394],
                       [    0,     0,     0,  ..., 11393, 11394, 11394]]),
       values=tensor([0.3333, 0.2357, 0.0758,  ..., 0.2500, 0.0874, 1.0000]),
       device='cuda:0', size=(11395, 11395), nnz=181877, layout=torch.sparse_coo)

In [5]:
model = Node2Vec(ppi._indices(), embedding_dim=16, walk_length=40,
                     context_size=5,  walks_per_node=10,
                     num_negative_samples=1, p=1, q=1, sparse=True).to(device)
model.cuda()

Node2Vec(11395, 16)

In [6]:
loader = model.loader(batch_size=128, shuffle=True)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.001)

In [ ]:
def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.cuda(), neg_rw.cuda())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

for epoch in range(1, 501):
    loss = train()
    if epoch%50 == 0:
        print (f"Epoch:{epoch}, The loss is {loss}")

model.eval()
str_fearures = model()

Epoch:50, The loss is 0.7416343397564358
Epoch:100, The loss is 0.7289802021450467
Epoch:150, The loss is 0.7278648290369246
Epoch:200, The loss is 0.7276557008425395
Epoch:250, The loss is 0.7271610988510979
Epoch:300, The loss is 0.7270283712281121
Epoch:350, The loss is 0.7272047466701932
Epoch:400, The loss is 0.727069103717804


In [41]:
feature = torch.load('feature.pt')
feature.x.to(device)

tensor([[0.0000, 0.0053, 0.0000,  ..., 0.0304, 0.0910, 0.0173],
        [0.0000, 0.0053, 0.0000,  ..., 0.0662, 0.3309, 0.0425],
        [0.0633, 0.0158, 0.0557,  ..., 0.2257, 0.3389, 0.5127],
        ...,
        [0.0000, 0.0053, 0.0000,  ..., 0.1495, 0.0753, 0.1055],
        [0.0000, 0.0053, 0.0000,  ..., 0.0920, 0.0548, 0.1508],
        [0.0000, 0.0106, 0.0000,  ..., 0.0452, 0.0000, 0.0000]],
       device='cuda:0')

In [42]:
feature

Data(x=[11395, 48])

In [43]:
combined_tensor = torch.cat((str_fearures, feature.x.to(device)), dim=1)  # dim=1 表示按列合并

In [44]:
combined_tensor.size()

torch.Size([11395, 64])

In [45]:
torch.save(combined_tensor, "feature_with_rw30.pt")